# cloud.google.container
This notebook interact with the google cloud container service

In [12]:
#import
import googleapiclient.discovery
from apiclient.discovery import build
import ipywidgets as widgets
from pandas import DataFrame
import json

In [6]:
# %load getCredentialsFromFile.py


def getCredentials():
    from  oauth2client import file
    import httplib2
    import ipywidgets as widgets
    print "Getting the credentials from file..."
    storage = file.Storage("oauth2.dat")
    credentials=storage.get()
    if credentials is None or credentials.invalid:
        print '❗'
        display(widgets.Valid(
            value=False,
            description='Credentials are ',
            disabled=False))
        display(widgets.HTML('go create a credential valid file here: <a target="_blank" href="cloud.google.auth.ipynb.ipynb">gcloud authorization notebook</a> and try again'))
    else:
        http_auth = credentials.authorize(httplib2.Http())
        print '✅ Ok'
        return credentials


In [7]:
credentials=getCredentials()

Getting the credentials from file...
✅ Ok


In [8]:
#create the services
from apiclient.discovery import build
container_service = build('container', 'v1', credentials=credentials)
resource_service = build('cloudresourcemanager', 'v1', credentials=credentials)

In [19]:
# %load chooseProjectId.py
#projectId is the variable that will contains the projectId that will be used in the API calls
projectId=None

#list the existing projects 
projects=resource_service.projects().list().execute()
#we create a dictionaray name:projectID foe a dropdown list widget
projectsList={project['name']:project['projectId'] for project in projects['projects']}
projectsList['None']='invalid'

#the dropdownlist widget
projectWidget=widgets.Dropdown(options=projectsList,description='Choose your Project',value='invalid')
#a valid widget that get valid when a project is selected
projectIdValid=widgets.Valid(value=False,description='')
display(widgets.Box([projectWidget,projectIdValid]))

def projectValueChange(sender):
    if projectWidget.value!='invalid':
        #when a valid project is selected ,the gloabl variable projectId is set 
        projectIdValid.value=True
        projectIdValid.description=projectWidget.value
        global projectId
        projectId=projectWidget.value    
    else:
        projectIdValid.value=False
        projectIdValid.description=''
projectWidget.observe(projectValueChange, 'value')

In [18]:
zone='europe-west2-b'

In [20]:
#call the container service to get the servConfig
serverConfig=container_service.projects().zones().getServerconfig(zone=zone,projectId=projectId).execute()

print(json.dumps(serverConfig, indent=2))

{
  "defaultClusterVersion": "1.6.4", 
  "validNodeVersions": [
    "1.6.6", 
    "1.6.4", 
    "1.5.7", 
    "1.4.9"
  ], 
  "validImageTypes": [
    "CONTAINER_VM", 
    "COS"
  ], 
  "validMasterVersions": [
    "1.6.6", 
    "1.6.4"
  ], 
  "defaultImageType": "COS"
}
